# Generate font size data visualization

This notebook generates a visualization for quotation data in which each word has font size proportional to the frequency of quotation.

In [1]:
# Probably some of these are irrelevant imports used for earlier visualizations

import sys
#print(sys.version)
import pandas as pd
import nltk
import math
import json
from ast import literal_eval
import numpy as np
import re
from IPython.core.display import HTML

3.11.4 (main, Jul  5 2023, 09:00:44) [Clang 14.0.6 ]


In [13]:
## Read in the path to the JSON file
name_of_setting = 't2-c3-n2-m3-no-stops'
df = pd.read_json(f'../data/{name_of_setting}.json')

In [16]:
df

,creator,datePublished,docSubType,docType,id,identifier,isPartOf,issueNumber,language,outputFormat,...,title,url,volumeNumber,wordCount,numMatches,Locations in A,Locations in B,abstract,keyphrase,subTitle
0,[Rainer Emig],2006-01-01,book-review,article,http://www.jstor.org/stable/41158244,"[{'name': 'issn', 'value': '03402827'}, {'name...",Amerikastudien / American Studies,3,[eng],"[unigram, bigram, trigram]",...,Review Article,http://www.jstor.org/stable/41158244,51,1109,1,"[[130022, 130046]]","[[6851, 6875]]",None,None,None
1,[Martin Green],1970-01-01,book-review,article,http://www.jstor.org/stable/3722819,"[{'name': 'issn', 'value': '00267937'}, {'name...",The Modern Language Review,1,[eng],"[unigram, bigram, trigram]",...,Review Article,http://www.jstor.org/stable/3722819,65,1342,0,[],[],None,None,None
2,[Richard Exner],1982-01-01,book-review,article,http://www.jstor.org/stable/40137021,"[{'name': 'issn', 'value': '01963570'}, {'name...",World Literature Today,1,[eng],"[unigram, bigram, trigram]",...,Review Article,http://www.jstor.org/stable/40137021,56,493,0,[],[],None,None,None
3,[Ruth Evelyn Henderson],1925-10-01,research-article,article,http://www.jstor.org/stable/802346,"[{'name': 'issn', 'value': '00138274'}, {'name...",The English Journal,8,[eng],"[unigram, bigram, trigram, fullText]",...,American Education Week--November 16-22; Some ...,http://www.jstor.org/stable/802346,14,2161,0,[],[],None,None,None
4,[Alan Palmer],2011-12-01,research-article,article,http://www.jstor.org/stable/10.5325/style.45.4...,"[{'name': 'issn', 'value': '00394238'}, {'name...",Style,4,[eng],"[unigram, bigram, trigram]",...,Rejoinder to Response by Marie-Laure Ryan,http://www.jstor.org/stable/10.5325/style.45.4...,45,1127,0,[],[],None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5879,[Michaela Giesenkirchen],2005-10-01,research-article,article,http://www.jstor.org/stable/27747183,"[{'name': 'issn', 'value': '15403084'}, {'name...",American Literary Realism,1,[eng],"[unigram, bigram, trigram]",...,Ethnic Types and Problems of Characterization ...,http://www.jstor.org/stable/27747183,38,7349,1,"[[23799, 24121]]","[[41472, 41793]]",None,None,None
5880,[Leon Botstein],2005-07-01,misc,article,http://www.jstor.org/stable/4123220,"[{'name': 'issn', 'value': '00274631'}, {'name...",The Musical Quarterly,2,[eng],"[unigram, bigram, trigram]",...,On the Power of Music,http://www.jstor.org/stable/4123220,88,1525,0,[],[],None,None,None
5881,[Linda M. Shires],2013-01-01,research-article,article,http://www.jstor.org/stable/24575734,"[{'name': 'issn', 'value': '10601503'}, {'name...",Victorian Literature and Culture,4,[eng],"[unigram, bigram, trigram]",...,"HARDY'S MEMORIAL ART: IMAGE AND TEXT IN ""WESSE...",http://www.jstor.org/stable/24575734,41,10736,1,"[[173657, 173756]]","[[33963, 34061]]",None,None,None
5882,[Edward H. Cohen],1990-07-01,misc,article,http://www.jstor.org/stable/3827815,"[{'name': 'issn', 'value': '00425222'}, {'name...",Victorian Studies,4,[eng],"[unigram, bigram, trigram]",...,Victorian Bibliography for 1989,http://www.jstor.org/stable/3827815,33,81819,0,[],[],None,None,None


In [17]:
df['year'] = pd.DatetimeIndex(df['datePublished']).year

In [18]:

df['Decade'] = df['year'] - (df['year'] % 10)

In [22]:
# Infer naming variables from path

textTitle = startData.rsplit("_", 4)[-3]
publicationYear = startData.rsplit("_", 4)[-4]
authorSurname = startData.rsplit("_", 4)[-5]
authorSurname = authorSurname.rsplit("/", 1)[-1]
hyperparSuffix = startData.rsplit("_", 4)[-1]
hyperparSuffix = f"_{hyperparSuffix[:-6]}"
dataDir = startData.rsplit("/", 4)[0]

print(f"Author surname: {authorSurname}\nPublication year: {publicationYear}\nText title: {textTitle}\nHyperparameters suffix: {hyperparSuffix}\nData directory:{dataDir}")

projectName = f"{authorSurname}_{publicationYear}_{textTitle}"
sourceDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/Source"
corpusDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/Corpus"
resultsDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/Results"

Author surname: Joyce
Publication year: 1922
Text title: Ulysses
Hyperparameters suffix: _t2-c3-n2-m3-nostops
Data directory:/Users/milan/Library/CloudStorage/GoogleDrive-mtt2126@columbia.edu/My Drive/iAnnotate/MIT/Quotable Content/Data


In [19]:
# Adapted from text-matcher
class Text: 
    def __init__(self, filename): 
        self.filename = filename
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        self.length = spans[-1][-1] 
        
    @property
    def text(self):
        """ Reads the file in memory. """
        f = open(self.filename, encoding='utf-8', errors='ignore')
        return f.read() 

    @property
    def tokens(self, removeStopwords=True): 
        """ Tokenizes the text, breaking it up into words, removing punctuation. """
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        self.length = spans[-1][-1] 
        tokens = tokenizer.tokenize(self.text)
        tokens = [ token.lower() for token in tokens ] # make them lowercase
        if not removeStopwords: 
            self.spans = spans
            return tokens
        tokenSpans = list(zip(tokens, spans)) # zip it up
        stopwords = nltk.corpus.stopwords.words('english') # get stopwords
        tokenSpans = [ token for token in tokenSpans if token[0] not in stopwords ] # remove stopwords from zip
        self.spans = [ x[1] for x in tokenSpans ] # unzip; get spans
        return [ x[0] for x in tokenSpans ] # unzip; get tokens

In [20]:
# Load source text

sourceText = "../middlemarch.txt"

text = Text(sourceText)

In [21]:
# Get the size of the text
textALength = text.length

# Make an empty array the size of the text
tally = np.zeros(textALength, dtype=int)

In [22]:
# Read the matched locations from the results dataset, and literally evaluate them into lists. 

locationsInA = df['Locations in A']

In [23]:
# Tally up every time a letter in the text is quoted. 
for article in locationsInA: 
    for locRange in article: 
         for i in range(locRange[0], min(locRange[1]+1, len(tally))):
                tally[i] += 1

In [24]:
# Make a font size list for all the values in the tally. 

font = list(np.arange(12,(tally.max()+13)))
fontList = font

In [25]:
# Create a CSS Stylesheet for each font value in the map. 
fontCSS = ""
for i, font in zip(range(0, tally.max()+1), fontList): 
    fontCSS += ".c-%s { font-size: %spx; }" % (i, font)

In [31]:
#n = 50
#
#checkpoints = np.linspace(0, textALength, n).round()
#checkpoints = [int(point) for point in checkpoints]

In [26]:
def span(val): 
    return '<span class="c-%s">' % val

previousVal = None
for i, valChar in enumerate(zip(tally, text.text)):
    val, char = valChar[0], valChar[1]
    if previousVal == None: 
        # First character. 
        out = '<span class="c-%s">' % val
    elif val != previousVal: 
        out += '</span><span class="c-%s">' % val
    out += char
    previousVal = val

In [28]:
html = """<!DOCTYPE html>
<html>
<head>
  <link href="https://fonts.googleapis.com/css?family=Raleway" rel="stylesheet"> 
  <style>
  main { 
      width: 600px; 
  }
  %s
  </style>
  """ % (fontCSS)
html += """
  </head>
  <body><main>%s</main></body></html>
  """ % (out)

In [29]:
# Add html paragraph tags (for text where paragraphing is indicated by an empty line, i.e. two line breaks)

html = re.sub("\n\n", "</p>\n\n<p>", html)

In [30]:
# Where no span class is specified at start of para, reiterate most recent span class

untaggedParas = len(re.findall(r'\n\n<p>([^<])', html))

while untaggedParas != 0:
    oldUntagged = untaggedParas
    html = re.sub(r'(<span class="c-\d">)([^<>]*)</p>\n\n<p>([^<])', r'\1\2</p>\n\n<p>\1\3', html)
    untaggedParas = len(re.findall(r'\n\n<p>([^<])', html))
    print(untaggedParas)
    if untaggedParas == oldUntagged:
        print(f"Done: {untaggedParas} paragraphs couldn't be corrected")
        break
    if untaggedParas == 0:
        print("Done: no untagged paragraphs remaining")
        break

3631
3032
2594
2238
1954
1721
1520
1356
1216
1098
1004
918
846
786
732
689
649
615
588
564
544
528
514
502
491
483
475
468
461
456
451
446
442
439
436
434
432
430
428
426
425
424
423
422
421
420
419
418
417
416
415
414
413
412
411
410
409
408
408
Done: 408 paragraphs couldn't be corrected


In [31]:
with open(f"../middlemarch_fontsizeviz-{name_of_setting}.html", 'w') as f: 
    f.write(html)
    f.close()

In [32]:
with open(f"../annotated_by_size.html", 'w') as f: 
    f.write(html)
    f.close()